In [1]:
# Import modules 
import scipy
import plotly
import numpy as np
import pandas as pd
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from sklearn import preprocessing, metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import scale
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats.distributions import randint 
plotly.tools.set_credentials_file(username='alexd123', api_key='R93O7RUpfb3q7fVqXKHQ')
from  plotly.offline import plot
from importlib import reload

# Import data
df_train = pd.read_csv(r'C:\data\titanic\all\train.csv')
df_test = pd.read_csv(r'C:\data\titanic\all\test.csv')

C:\Users\Student\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

C:\Users\Student\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



In [2]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
# Store target variable of training data
survived_train = df_train.Survived

# Concatenate training and test sets
data = pd.concat([df_train.drop(['Survived'], axis=1), df_test])

In [5]:
survived_train.head()   #boolean 

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [6]:
data.info() #important to do this to see missing values (nulls)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1046 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1308 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


In [7]:
#from this it is clear that there are missing values therefor we need to imput them 
#The median values will be used 
data['Age'] = data.Age.fillna(data.Age.median())
data['Fare'] = data.Fare.fillna(data.Fare.median())

In [8]:
data.info() #here we can see that the null vaues have been filled in with median values 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Name           1309 non-null object
Sex            1309 non-null object
Age            1309 non-null float64
SibSp          1309 non-null int64
Parch          1309 non-null int64
Ticket         1309 non-null object
Fare           1309 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 122.7+ KB


In [9]:
#now we need to encode our data to get rid of the male and female for 1 and 0 ect
data = pd.get_dummies(data, columns=['Sex'], drop_first=True) #the drop part removes the sex_female column as they are giving/encoding the same information 

In [10]:
data.head() #this shows that the sex column has been changed for 1 and 0 to represent male and female

,PassengerId,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_male
0,1,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,0
4,5,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,1


In [11]:
#now we are going to select the columns we want to use in our machine learning model
data = data[['Sex_male', 'Fare', 'Age','Pclass', 'SibSp']]
data.head()

,Sex_male,Fare,Age,Pclass,SibSp
0,1,7.2500,22.0,3,1
1,0,71.2833,38.0,1,1
2,0,7.9250,26.0,3,0
3,0,53.1000,35.0,1,1
4,1,8.0500,35.0,3,0


In [12]:
data.info()  #we can see that all of our entries are now full. 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 5 columns):
Sex_male    1309 non-null uint8
Fare        1309 non-null float64
Age         1309 non-null float64
Pclass      1309 non-null int64
SibSp       1309 non-null int64
dtypes: float64(2), int64(2), uint8(1)
memory usage: 52.4 KB


In [13]:
#split the document back into train and test datasets 
data_train = data.iloc[:891]
data_test = data.iloc[891:]

In [14]:
X = data_train.values
test = data_test.values
y = survived_train.values #values turnes it into a numpy array 

In [15]:
X = scale(X) #scale the data to reduce the influence of much bigger numbers 

In [16]:
# KNeighborsClassifier

In [17]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [18]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, random_state=11)

In [19]:
ytest

array([1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 0, 0], dtype=int64)

In [20]:
Xtrain.shape

(668, 5)

In [21]:
neigh.fit(Xtrain,ytrain)
ypred = neigh.predict(Xtest)

In [22]:
accuracy = metrics.accuracy_score(ytest, ypred)

In [23]:
accuracy

0.8340807174887892

In [24]:
f1 = metrics.f1_score(ytest, ypred)

In [25]:
f1

0.7375886524822695

In [26]:
#  DecisionTreeClassifier

In [27]:
dtc = DecisionTreeClassifier(max_depth=3)

In [28]:
dtc.fit(Xtrain,ytrain)
ypred = dtc.predict(Xtest)

In [29]:
accuracy2 = metrics.accuracy_score(ytest, ypred)

In [30]:
accuracy2

0.8699551569506726

In [31]:
header = data.columns.values

In [32]:
header

array(['Sex_male', 'Fare', 'Age', 'Pclass', 'SibSp'], dtype=object)

In [33]:
i = dtc.feature_importances_

In [34]:
dict(zip(header,dtc.feature_importances_))

{'Sex_male': 0.5939016275707054,
 'Fare': 0.04432193202633814,
 'Age': 0.07629975807190904,
 'Pclass': 0.23889902314903502,
 'SibSp': 0.046577659182012496}

In [35]:
names = header

In [36]:
data = [
    Bar(x = i,
       y=names,
       orientation = 'h'
    )
]
layout = Layout(
    xaxis = dict(title = 'Importance of Features'),
    yaxis = dict(title = 'Features'),
    width = 800,
    margin = Margin(
    l=250,
    r=50,
    b=100,
    t=50,
    pad=4)
)
fig = dict(data=data,layout=layout)
py.iplot(fig)